In [71]:
from bs4 import BeautifulSoup
import requests
from lxml import etree
import csv
from datetime import datetime

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }


LINKS_FOR_NATION_THRESHOLD=400

NATIONS_TO_GET=('/united-states','/united-kingdom','/italy','/estonia')




fields_W_tradeField=['Name','Trade Name', 'Address', 'City', 'State', 'Region', 'Country','Location Type']
fields_no_tradeField=['Name', 'Address', 'City', 'State', 'Region', 'Country','Location Type']



path_dataset_aziende='info-clipper-ds.csv'

path_dataset_link='sites_per_country.csv'

path_tempi_estrazione='info-clipper-tempiEstrazione.txt'

#acquisizione link
get_links(path_dataset_link, path_tempi_estrazione)
scrape_data(path_dataset_link, path_dataset_aziende)




Address
j: 0
XPAth: //*[@id="company_identity"]/div[1]/div[1]/span/span
VAL XPath //*[@id="company_identity"]/div[1]/div[1]/span/strong
Nome: Name
valore: MASIMO CORPORATION
atteso: Name
corretto True
j: 1
XPAth: //*[@id="company_identity"]/div[1]/div[2]/span/span
VAL XPath //*[@id="company_identity"]/div[1]/div[2]/span/text()
Nome: Address
valore: 52 DISCOVERY
atteso: Address
corretto True
j: 2
XPAth: //*[@id="company_identity"]/div[1]/div[3]/span/span
VAL XPath //*[@id="company_identity"]/div[1]/div[3]/span/text()
XPAth: //*[@id="company_identity"]/div[1]/div[3]/span/span
VAL XPath //*[@id="company_identity"]/div[1]/div[3]/span/text()
Nome: <Element span at 0x7fc7d496cd80>
valore:               926183105
city: IRVINE
atteso: City
corretto True
j: 3
XPAth: //*[@id="company_identity"]/div[1]/div[4]/span/span
XPAth2: //*[@id="company_identity"]/div[1]/div[5]/span/span
VAL XPath //*[@id="company_identity"]/div[1]/div[4]/span/text()
VAL XPath //*[@id="company_identity"]/div[1]/div[5]/span

NameError: ignored

#SCRAPING DEI LINK DELLE PAGINE DI DETTAGLIO


In [ ]:
#Acquisizione dei link verso le pagine di dettaglio

def get_links_to_companys(links4nation,nation, url):
    
    
    req = requests.get(url, headers)
    
    
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    links = dom.xpath('//*[@id="leContenu"]/div[1]')
    conta=0
    list_links=[]
    for a in soup.find_all('a', href = True):
        
        if '/company' in a['href'] and '/search' not in a['href']:
          
            
            conta+=1
            list_links.append(a['href'])

  

    links4nation[nation]=list_links

    
    


In [ ]:
#preparazione per creazione di un dataframe
#Adatta il dizionario ad avere valori aventi la stessa lunghezza
def adapt_links(links):

  out=dict()
  for k in links:
    list_links=links[k]
    if len(list_links)>LINKS_FOR_NATION_THRESHOLD:
      out[k]=list_links[2:LINKS_FOR_NATION_THRESHOLD+3]
    else:
      while(len(list_links)<=LINKS_FOR_NATION_THRESHOLD+3):
        list_links.append(None)
      out[k]=list_links[2:LINKS_FOR_NATION_THRESHOLD+3]
  return out

In [ ]:
#stampa su csv dei link delle pagine di dettaglio per country
#Stampa sul csv tramite pandas
import pandas as pd
def save_links(links, csvFile):
  df=pd.DataFrame.from_dict(links)
  df.to_csv(path_or_buf=path_dataset_link,index=False)



In [ ]:
#metodo ausialiare per il calcolo del tempo impiegato per l'acquisizione del dato via xpath
def get_time_ex(start, k , end):
    
    time=end-start
    duration_in_s = time.total_seconds() 
    time_string=k+': '+str(duration_in_s)
    return time_string

In [ ]:
def get_links(path_dataset_link, path_tempi_estrazione): 
    links_for_country={
                    'USA': 'https://www.info-clipper.com/en/companies/united-states.us/groups.html', 
                     'UK':'https://www.info-clipper.com/en/companies/united-kingdom.gb/groups.html', 
                     'Italy': 'https://www.info-clipper.com/en/companies/italy.it/groups.html', 
                     'Estony':'https://www.info-clipper.com/en/companies/estonia.ee/groups.html'}
    dataset_links = open(path_dataset_link, 'w')
    tempi_estrazione_link=open(path_tempi_estrazione,'w')


    dataset_links= csv.writer(dataset_links)
    

    d = {}
    id_ = 0

    #creazione di un csv con tutti i link

    links=dict()
    for k in links_for_country:
        url=links_for_country[k]
        start=datetime.now()
        get_links_to_companys(links,k,url)
        end=datetime.now()

        time_string=get_time_ex(start, k , end)
        tempi_estrazione_link.write(time_string+'\n')

    links=adapt_links(links)
    
    save_links(links,dataset_links)

    tempi_estrazione_link.close()                  

###SCRAPING DATI
###USA ha una struttura diversa e potrebbe esserci un campo in più
###UK, Italia, Estonia hanno una struttura diversas da USA


---



In [70]:
from csv import reader
from csv import writer
def scrape_data(path_dataset_link, path_dataset_aziende):
  with open(path_dataset_link, 'r') as read_obj:
   
    csv_reader = reader(read_obj)
    
    writer_a=open(path_dataset_aziende,'w')
    csv_writer=csv.writer(writer_a)
    id=1
    headers=['URL', 'ID', 'Name', 'corretto Name', 'tempo Estrazione Name',
             'Trade Name', 'corretto Trade Name', 'tempo estrazione Trade Name',
             'Address Name', 'corretto Address', 'tempo estrazione Address',
             'City', 'corretto City', 'tempo estrazione City',
             'Postalcode',
             'State', 'corretto State', 'tempo estrazione State',
             'Country', 'corretto Country', 'tempo estrazione Country',
             'Location type', 'corretto location type', 'tempo estrazione Location Type',
             ]
    
    csv_writer.writerow(headers)
    conta=0
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
      if conta>0:
        # row variable is a list that represents a row in csv
        for j in range (0,4):
          if j==0:
            link=row[j]
            infos=extract_info_for_USA(link,id)
          else:
            if row[j]!='':
              link=row[j]
              infos=extract_info_for_other_countries(link,id)

          csv_writer.writerow(infos)
          id+=1
      conta+=1
    writer_a.close()
          


In [ ]:
def extract_info_for_USA(link,id):
    x_paths_W_tradeFields=[
        ('//*[@id="company_identity"]/div[1]/div[1]/span/span','//*[@id="company_identity"]/div[1]/div[1]/span/strong'),
             ('//*[@id=\"company_identity\"]/div[1]/div[2]/span/span','//*[@id=\"company_identity\"]/div[1]/div[2]/span/text()'),
             
             ('//*[@id=\"company_identity\"]/div[1]/div[3]/span/span','//*[@id=\"company_identity\"]/div[1]/div[3]/span/text()'),
             
             ('//*[@id=\"company_identity\"]/div[1]/div[4]/span/span','//*[@id=\"company_identity\"]/div[1]/div[4]/span/text()'),
             ('//*[@id=\"company_identity\"]/div[1]/div[5]/span/span','//*[@id=\"company_identity\"]/div[1]/div[5]/span/text()'),
             ('//*[@id=\"company_identity\"]/div[1]/div[6]/span/span','//*[@id=\"company_identity\"]/div[1]/div[6]/span/text()'),
             ('//*[@id=\"company_identity\"]/div[1]/div[7]/span/span','//*[@id="company_identity"]/div[1]/div[7]/span/a'),
             ('//*[@id=\"company_identity\"]/div[1]/div[8]/span/span','//*[@id=\"company_identity\"]/div[1]/div[8]/span/text()')]
             
    x_paths_no_tradeFields=[
            ('//*[@id="company_identity"]/div[1]/div[1]/span/span','//*[@id="company_identity"]/div[1]/div[1]/span/strong'),
             ('//*[@id=\"company_identity\"]/div[1]/div[2]/span/span','//*[@id=\"company_identity\"]/div[1]/div[2]/span/text()'),
             
             ('//*[@id=\"company_identity\"]/div[1]/div[3]/span/span','//*[@id=\"company_identity\"]/div[1]/div[3]/span/text()'),
            
             ('//*[@id=\"company_identity\"]/div[1]/div[4]/span/span','//*[@id=\"company_identity\"]/div[1]/div[4]/span/text()'),
             ('//*[@id=\"company_identity\"]/div[1]/div[5]/span/span','//*[@id=\"company_identity\"]/div[1]/div[5]/span/text()'),
             ('//*[@id=\"company_identity\"]/div[1]/div[6]/span/span','//*[@id="company_identity"]/div[1]/div[6]/span/a'),
             ('//*[@id=\"company_identity\"]/div[1]/div[7]/span/span','//*[@id=\"company_identity\"]/div[1]/div[7]/span/text()')]


    #i fields assocaiti
    #servono per un check
    fields_W_tradeField=['Name','Trade Name', 'Address', 'City', 'State', 'Region', 'Country','Location Type']
    fields_no_tradeField=['Name', 'Address', 'City', 'State', 'Region', 'Country','Location Type']
    
    information_schema=[]
    start = datetime.now()
    req = requests.get(link, headers)
    end = datetime.now()
    time_diff=get_time_ex(start,'access',end)
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    contains_trade_name=False
    #verifico la presenza di Trade Name
    xPathColumn, _=x_paths_W_tradeFields[1]
    nameField=dom.xpath(xPathColumn)[0].text
    print(nameField)
    if 'Trade' in nameField:
      x_paths=x_paths_W_tradeFields
      fields=fields_W_tradeField
      infos=extractDataWithNameField(dom,x_paths, fields,link,id, information_schema)
      
    else:
      x_paths=x_paths_no_tradeFields
      fields=fields_no_tradeField
      
      infos=extractDataWithNoNameField(dom,x_paths, fields,link,id,information_schema)

    return infos
   
    

In [66]:
def extract_info_for_other_countries(link, id):
    
    
    x_paths_W_tradeFields=[
        ('//*[@id="company_identity"]/div[1]/div[1]/span/span','//*[@id="company_identity"]/div[1]/div[1]/span/strong'),
             ('//*[@id=\"company_identity\"]/div[1]/div[2]/span/span','//*[@id=\"company_identity\"]/div[1]/div[2]/span/text()'),
             
             ('//*[@id=\"company_identity\"]/div[1]/div[3]/span/span','//*[@id=\"company_identity\"]/div[1]/div[3]/span/text()'),
             ('//*[@id=\"company_identity\"]/div[1]/div[4]/span/span','//*[@id=\"company_identity\"]/div[1]/div[4]/span/text()'),
             
             ('//*[@id=\"company_identity\"]/div[1]/div[5]/span/span','//*[@id="company_identity"]/div[1]/div[5]/span/a'),
             ('//*[@id=\"company_identity\"]/div[1]/div[6]/span/span','//*[@id=\"company_identity\"]/div[1]/div[6]/span/text()')]
             
    x_paths_no_tradeFields=[
            ('//*[@id="company_identity"]/div[1]/div[1]/span/span','//*[@id="company_identity"]/div[1]/div[1]/span/strong'),
             ('//*[@id=\"company_identity\"]/div[1]/div[2]/span/span','//*[@id=\"company_identity\"]/div[1]/div[2]/span/text()'),
             
             ('//*[@id=\"company_identity\"]/div[1]/div[3]/span/span','//*[@id=\"company_identity\"]/div[1]/div[3]/span/text()'),
            

             ('//*[@id=\"company_identity\"]/div[1]/div[4]/span/span','//*[@id="company_identity"]/div[1]/div[4]/span/a'),
             ('//*[@id=\"company_identity\"]/div[1]/div[5]/span/span','//*[@id=\"company_identity\"]/div[1]/div[5]/span/text()')]

    fields_W_tradeField=['Name','Trade Name', 'Address', 'City', 'Country','Location Type']
    ields_no_tradeField=['Name', 'Address', 'City', 'Country','Location Type']
  

    information_schema=[]
    start = datetime.now()
    req = requests.get(link, headers)
    end = datetime.now()
    time_diff=get_time_ex(start,'access',end)
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    contains_trade_name=False
    #verifico la presenza di Trade Name
    xPathColumn, _=x_paths_W_tradeFields[1]
    nameField=dom.xpath(xPathColumn)[0].text
    print(nameField)
    if 'Trade' in nameField:
      x_paths=x_paths_W_tradeFields
      fields=fields_W_tradeField
      infos=extractDataWithNameField_NO_USA(dom,x_paths, fields,link,id, information_schema)
      
    else:
      x_paths=x_paths_no_tradeFields
      fields=fields_no_tradeField
      
      infos=extractDataWith_NO_NameField_NO_USA(dom,x_paths, fields,link,id,information_schema)

    return infos

In [ ]:
def extractDataWithNoNameField(dom, x_paths, fields, link, id,information_schema):
    information_schema.append(link)
    information_schema.append(id)
    row=[]
    
    x_path_nameCity='//*[@id="company_identity"]/div[1]/div[4]/span/a'
    x_path_nameCountry='//*[@id="company_identity"]/div[1]/div[5]/span/a'
    for j in range(0,len(fields)):
        
        #name
        print('j:',j)
        correct=True
        nameFieldXpath, nameValueXpath=x_paths[j]
       

        #ESTRAZIONE POSTAL CODE
        #nome
        if j==0:
          start = datetime.now()
          nameField=dom.xpath(nameFieldXpath)[0].text
          nameValue=dom.xpath(nameValueXpath)[0].text
          end = datetime.now()
          time_diff=get_time_ex(start,'_',end)
          if nameField!=fields[j]:
            
            correct=False

          information_schema.append(nameValue)
          
          information_schema.append(correct)
          information_schema.append(time_diff)
          
          print("XPAth:", nameFieldXpath)
          print("VAL XPath", nameValueXpath)
          print('Nome:',nameField)
          print('valore:',nameValue)
          print('atteso:',fields[j])
          print('corretto', correct)



       
        #address      
        elif j==1:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              information_schema.append(None)
              information_schema.append('Address' in nameField)
              information_schema.append(0)

              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

        #city
        elif j==2:
              x_path_nameCity='//*[@id="company_identity"]/div[1]/div[3]/span/a'
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0]
              nameValue=dom.xpath(nameValueXpath)[1].replace('\n','').replace('-','').replace('\t','')
              
              nameCity=dom.xpath(x_path_nameCity)[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              
              
              
              information_schema.append(nameCity)
              
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              information_schema.append(nameValue)

              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('city:',nameCity)
              print('atteso:',fields[j])
              print('corretto', correct)

        elif j==3:
              regionColXPath,regionValXPath=x_paths[j+1]
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              nameFieldState=dom.xpath(regionColXPath)[0].text
              nameValueState=dom.xpath(regionValXPath)[0]
              
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              
              
              
              
              
              information_schema.append(nameValue+'('+nameValueState+')')
              
              information_schema.append((nameField==fields[j]) and (nameFieldState==fields[j+1]))
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("XPAth2:", regionColXPath)
              print("VAL XPath", nameValueXpath)
              print("VAL XPath", regionValXPath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              
              print('valore 2:', nameValueState)
              print('valore inserito:',nameValue+'('+nameValueState+')')
              print('atteso:',fields[j])
              print('atteso2:',fields[j+1])
              print('corretto', (nameField==fields[j]) and (nameFieldState==fields[j+1]))


        

        elif j==5:
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
        elif j==6:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
             
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

    return information_schema

              
          
          
       
          


In [63]:
def extractDataWithNameField_NO_USA(dom, x_paths,fields,link, id, information_schema):
    information_schema.append(link)
    information_schema.append(id)
    
    
    x_path_nameCity='//*[@id="company_identity"]/div[1]/div[4]/span/a'
    x_path_nameCountry='//*[@id="company_identity"]/div[1]/div[5]/span/a'
    for j in range(0,len(fields)):
        
        #name
        print('j:',j)
        correct=True
        nameFieldXpath, nameValueXpath=x_paths[j]
       

        #ESTRAZIONE POSTAL CODE
        #nome
        if j==0:
          start = datetime.now()
          nameField=dom.xpath(nameFieldXpath)[0].text
          nameValue=dom.xpath(nameValueXpath)[0].text
          end = datetime.now()
          time_diff=get_time_ex(start,'_',end)
          if nameField!=fields[j]:
            
            correct=False

          information_schema.append(nameValue) 
          information_schema.append(correct)
          information_schema.append(time_diff)
          
          print("XPAth:", nameFieldXpath)
          print("VAL XPath", nameValueXpath)
          print('Nome:',nameField)
          print('valore:',nameValue)
          print('atteso:',fields[j])
          print('corretto', correct)



        #trade name
        elif j==1:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

           
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
              
        #address      
        elif j==2:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

           
              information_schema.append(nameValue)
              information_schema.append(fields[j]==nameField)
              information_schema.append(time_diff)
             
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

        #city
        elif j==3:
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[1].replace('\n','').replace('-','').replace('\t','')
              nameCity=dom.xpath(x_path_nameCity)[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              x_path_nameCity='//*[@id="company_identity"]/div[1]/div[4]/span/a'
              
              
              information_schema.append(nameCity)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              information_schema.append(nameValue)

              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('city:',nameCity)
              print('atteso:',fields[j])
              print('corretto', correct)

       

        

        elif j==4:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              #state
              information_schema.append(nameValue)
              information_schema.append(True)
              information_schema.append(0)
              
              #country
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)

              
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
       

    
    return information_schema


In [ ]:
def extractDataWithNoNameField(dom, x_paths, fields, link, id,information_schema):
      information_schema.append(link)
      information_schema.append(id)
   
    
      x_path_nameCity='//*[@id="company_identity"]/div[1]/div[4]/span/a'
      x_path_nameCountry='//*[@id="company_identity"]/div[1]/div[5]/span/a'
      for j in range(0,len(fields)):
        
        #name
        print('j:',j)
        correct=True
        nameFieldXpath, nameValueXpath=x_paths[j]
       

        #ESTRAZIONE POSTAL CODE
        #nome
        if j==0:
          start = datetime.now()
          nameField=dom.xpath(nameFieldXpath)[0].text
          nameValue=dom.xpath(nameValueXpath)[0].text
          end = datetime.now()
          time_diff=get_time_ex(start,'_',end)
          if nameField!=fields[j]:
            
            correct=False

          information_schema.append(nameValue)
          
          information_schema.append(correct)
          information_schema.append(time_diff)
          
          print("XPAth:", nameFieldXpath)
          print("VAL XPath", nameValueXpath)
          print('Nome:',nameField)
          print('valore:',nameValue)
          print('atteso:',fields[j])
          print('corretto', correct)



       
        #address      
        elif j==1:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              information_schema.append(None)
              information_schema.append('Address' in nameField)
              information_schema.append(0)

              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

        #city
        elif j==2:
              x_path_nameCity='//*[@id="company_identity"]/div[1]/div[3]/span/a'
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0]
              nameValue=dom.xpath(nameValueXpath)[1].replace('\n','').replace('-','').replace('\t','')
              
              nameCity=dom.xpath(x_path_nameCity)[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              
              
              
              information_schema.append(nameCity)
              
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              information_schema.append(nameValue)

              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('city:',nameCity)
              print('atteso:',fields[j])
              print('corretto', correct)

        elif j==3:
              regionColXPath,regionValXPath=x_paths[j+1]
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              nameFieldState=dom.xpath(regionColXPath)[0].text
              nameValueState=dom.xpath(regionValXPath)[0]
              
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              
              
              
              
              
              information_schema.append(nameValue+'('+nameValueState+')')
              
              information_schema.append((nameField==fields[j]) and (nameFieldState==fields[j+1]))
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("XPAth2:", regionColXPath)
              print("VAL XPath", nameValueXpath)
              print("VAL XPath", regionValXPath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              
              print('valore 2:', nameValueState)
              print('valore inserito:',nameValue+'('+nameValueState+')')
              print('atteso:',fields[j])
              print('atteso2:',fields[j+1])
              print('corretto', (nameField==fields[j]) and (nameFieldState==fields[j+1]))


        

        elif j==5:
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)
              if type(nameValue)!=list:
                nameFieldXpath, nameValueXpath=x_paths[j-1]
                nameField=dom.xpath(nameFieldXpath)[0].text
                nameValue=dom.xpath(nameValueXpath)[0].text
              else:
                nameValue=nameValue[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
        elif j==6:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
             
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

      print(information_schema)
      return information_schema

              
          
          
       
          


In [ ]:
def extractDataWithNameField(dom, x_paths, fields, link, id,information_schema):
    information_schema.append(link)
    information_schema.append(id)
    
    
    x_path_nameCity='//*[@id="company_identity"]/div[1]/div[4]/span/a'
    x_path_nameCountry='//*[@id="company_identity"]/div[1]/div[7]/span/a'
    for j in range(0,len(fields)):
        
        #name
        print('j:',j)
        correct=True
        nameFieldXpath, nameValueXpath=x_paths[j]
       

        #ESTRAZIONE POSTAL CODE
        #nome
        if j==0:
          start = datetime.now()
          nameField=dom.xpath(nameFieldXpath)[0].text
          nameValue=dom.xpath(nameValueXpath)[0].text
          end = datetime.now()
          time_diff=get_time_ex(start,'_',end)
          if nameField!=fields[j]:
            
            correct=False

          information_schema.append(nameValue)
          information_schema.append(correct)
          information_schema.append(time_diff)
          
          print("XPAth:", nameFieldXpath)
          print("VAL XPath", nameValueXpath)
          print('Nome:',nameField)
          print('valore:',nameValue)
          print('atteso:',fields[j])
          print('corretto', correct)



        #trade name
        elif j==1:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

           
              information_schema.append(nameValue)
              information_schema.append(correct)
              information_schema.append(time_diff)
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
              
        #address      
        elif j==2:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              correct=nameField==fields[j]
              information_schema.append(nameValue)
              information_schema.append(correct)
              information_schema.append(time_diff)
  
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

        #city
        elif j==3:
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[1].replace('\n','').replace('-','').replace('\t','')
              nameCity=dom.xpath(x_path_nameCity)[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              x_path_nameCity='//*[@id="company_identity"]/div[1]/div[4]/span/a'
              
              
              information_schema.append(nameCity)
              
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              information_schema.append(nameValue)


              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('city:',nameCity)
              print('atteso:',fields[j])
              print('corretto', correct)


        #state and region
        elif j==4:
              regionColXPath,regionValXPath=x_paths[j+1]
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              nameFieldState=dom.xpath(regionColXPath)[0].text
              nameValueState=dom.xpath(regionValXPath)[0]
              
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              
              
              
              
              
              information_schema.append(nameValue+'('+nameValueState+')')
              
              information_schema.append((nameField==fields[j]) and (nameFieldState==fields[j+1]))
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("XPAth2:", regionColXPath)
              print("VAL XPath", nameValueXpath)
              print("VAL XPath", regionValXPath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              
              print('valore 2:', nameValueState)
              print('valore inserito:',nameValue+'('+nameValueState+')')
              print('atteso:',fields[j])
              print('atteso2:',fields[j+1])
              print('corretto', (nameField==fields[j]) and (nameFieldState==fields[j+1]))


        

        elif j==6:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)
              if type(nameValue)!=list:
                nameFieldXpath, nameValueXpath=x_paths[j-1]
                nameField=dom.xpath(nameFieldXpath)[0].text
                nameValue=dom.xpath(nameValueXpath)[0].text
              else:
                nameValue=nameValue[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
        elif j==7:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
             
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

    
    return information_schema

              
          
          
       
          

In [ ]:
def extractDataWithNameField(dom, x_paths, fields, link, id,information_schema):
    information_schema.append(link)
    information_schema.append(id)
    
    
    x_path_nameCity='//*[@id="company_identity"]/div[1]/div[4]/span/a'
    x_path_nameCountry='//*[@id="company_identity"]/div[1]/div[7]/span/a'
    for j in range(0,len(fields)):
        
        #name
        print('j:',j)
        correct=True
        nameFieldXpath, nameValueXpath=x_paths[j]
       

        #ESTRAZIONE POSTAL CODE
        #nome
        if j==0:
          start = datetime.now()
          nameField=dom.xpath(nameFieldXpath)[0].text
          nameValue=dom.xpath(nameValueXpath)[0].text
          end = datetime.now()
          time_diff=get_time_ex(start,'_',end)
          if nameField!=fields[j]:
            
            correct=False

          information_schema.append(naveValue)
          information_schema.append(correct)
          information_schema.append(time_diff)
          
          print("XPAth:", nameFieldXpath)
          print("VAL XPath", nameValueXpath)
          print('Nome:',nameField)
          print('valore:',nameValue)
          print('atteso:',fields[j])
          print('corretto', correct)



        #trade name
        elif j==1:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

           
              information_schema.append(naveValue)
              information_schema.append(correct)
              information_schema.append(time_diff)
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
              
        #address      
        elif j==2:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              correct=nameField==fields[j]
              information_schema.append(naveValue)
              information_schema.append(correct)
              information_schema.append(time_diff)
  
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

        #city
        elif j==3:
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[1].replace('\n','').replace('-','').replace('\t','')
              nameCity=dom.xpath(x_path_nameCity)[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              x_path_nameCity='//*[@id="company_identity"]/div[1]/div[4]/span/a'
              
              
              information_schema.append(nameCity)
              
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              information_schema.append(nameValue)


              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('city:',nameCity)
              print('atteso:',fields[j])
              print('corretto', correct)


        #state and region
        elif j==4:
              regionColXPath,regionValXPath=x_paths[j+1]
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              nameFieldState=dom.xpath(regionColXPath)[0].text
              nameValueState=dom.xpath(regionValXPath)[0]
              
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              
              
              
              
              
              information_schema.append(nameValue+'('+nameValueState+')')
              
              information_schema.append((nameField==fields[j]) and (nameFieldState==fields[j+1]))
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("XPAth2:", regionColXPath)
              print("VAL XPath", nameValueXpath)
              print("VAL XPath", regionValXPath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              
              print('valore 2:', nameValueState)
              print('valore inserito:',nameValue+'('+nameValueState+')')
              print('atteso:',fields[j])
              print('atteso2:',fields[j+1])
              print('corretto', (nameField==fields[j]) and (nameFieldState==fields[j+1]))


        

        elif j==6:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
        elif j==7:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
             
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

    
    return information_schema

              
          
          
       
          

In [68]:
def extractDataWith_NO_NameField_NO_USA(dom, x_paths,fields,link, id, information_schema):
    information_schema.append(link)
    information_schema.append(id)
    
    
    x_path_nameCity='//*[@id="company_identity"]/div[1]/div[3]/span/a'
    x_path_nameCountry='//*[@id="company_identity"]/div[1]/div[4]/span/a'
    for j in range(0,5):
        
        #name
        print('j:',j)
        correct=True
        nameFieldXpath, nameValueXpath=x_paths[j]
       

        #ESTRAZIONE POSTAL CODE
        #nome
        if j==0:
          start = datetime.now()
          nameField=dom.xpath(nameFieldXpath)[0].text
          nameValue=dom.xpath(nameValueXpath)[0].text
          end = datetime.now()
          time_diff=get_time_ex(start,'_',end)
          if nameField!=fields[j]:
            
            correct=False

          information_schema.append(nameValue)
          key_corretto='Correct '+ fields[j]
          key_tempo= 'Tempo estrazione' + fields[j] 
          information_schema.append(correct)
          information_schema.append(time_diff)
          
          print("XPAth:", nameFieldXpath)
          print("VAL XPath", nameValueXpath)
          print('Nome:',nameField)
          print('valore:',nameValue)
          print('atteso:',fields[j])
          print('corretto', correct)



      
              
        #address      
        elif j==1:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              information_schema.append(None)
              information_schema.append('Address' in nameField)
              information_schema.append(0)
              information_schema.append(nameValue)
              information_schema.append(correct)
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

        #city
        elif j==2:
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[1].replace('\n','').replace('-','').replace('\t','')
              nameCity=dom.xpath(x_path_nameCity)[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              x_path_nameCity='//*[@id="company_identity"]/div[1]/div[4]/span/a'
              
              
              information_schema.append(nameCity)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              information_schema.append(nameValue)
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('city:',nameCity)
              print('atteso:',fields[j])
              print('corretto', correct)

       

        

        elif j==3:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              information_schema.append(nameValue)
              information_schema.append(True)
              information_schema.append(0)
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)

             
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
        elif j==4:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
             
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

    return information_schema

In [ ]:
def extractDataWithNameField(dom, x_paths, fields, link, id,information_schema):
      information_schema.append(link)
      information_schema.append(id)
    
    
      x_path_nameCity='//*[@id="company_identity"]/div[1]/div[4]/span/a'
      x_path_nameCountry='//*[@id="company_identity"]/div[1]/div[7]/span/a'
      for j in range(0,len(fields)):
        
        #name
        print('j:',j)
        correct=True
        nameFieldXpath, nameValueXpath=x_paths[j]
       

        #ESTRAZIONE POSTAL CODE
        #nome
        if j==0:
          start = datetime.now()
          nameField=dom.xpath(nameFieldXpath)[0].text
          nameValue=dom.xpath(nameValueXpath)[0].text
          end = datetime.now()
          time_diff=get_time_ex(start,'_',end)
          if nameField!=fields[j]:
            
            correct=False

          information_schema.append(nameValue)
          information_schema.append(correct)
          information_schema.append(time_diff)
          
          print("XPAth:", nameFieldXpath)
          print("VAL XPath", nameValueXpath)
          print('Nome:',nameField)
          print('valore:',nameValue)
          print('atteso:',fields[j])
          print('corretto', correct)



        #trade name
        elif j==1:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

           
              information_schema.append(nameValue)
              information_schema.append(correct)
              information_schema.append(time_diff)
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
              
        #address      
        elif j==2:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              correct=nameField==fields[j]
              information_schema.append(nameValue)
              information_schema.append(correct)
              information_schema.append(time_diff)
  
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)

        #city
        elif j==3:
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[1].replace('\n','').replace('-','').replace('\t','')
              nameCity=dom.xpath(x_path_nameCity)[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)

              x_path_nameCity='//*[@id="company_identity"]/div[1]/div[4]/span/a'
              
              
              information_schema.append(nameCity)
              
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              information_schema.append(nameValue)


              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('city:',nameCity)
              print('atteso:',fields[j])
              print('corretto', correct)


        #state and region
        elif j==4:
              regionColXPath,regionValXPath=x_paths[j+1]
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              nameFieldState=dom.xpath(regionColXPath)[0].text
              nameValueState=dom.xpath(regionValXPath)[0]
              
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              
              
              
              
              
              information_schema.append(nameValue+'('+nameValueState+')')
              
              information_schema.append((nameField==fields[j]) and (nameFieldState==fields[j+1]))
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("XPAth2:", regionColXPath)
              print("VAL XPath", nameValueXpath)
              print("VAL XPath", regionValXPath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              
              print('valore 2:', nameValueState)
              print('valore inserito:',nameValue+'('+nameValueState+')')
              print('atteso:',fields[j])
              print('atteso2:',fields[j+1])
              print('corretto', (nameField==fields[j]) and (nameFieldState==fields[j+1]))


        

        elif j==6:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)
              if type(nameValue)!=list:
                nameFieldXpath, nameValueXpath=x_paths[j-1]
                nameField=dom.xpath(nameFieldXpath)[0].text
                nameValue=dom.xpath(nameValueXpath)[0].text
              else:
                nameValue=nameValue[0].text
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
        elif j==7:
              start = datetime.now()
              nameField=dom.xpath(nameFieldXpath)[0].text
              nameValue=dom.xpath(nameValueXpath)[0]
              end = datetime.now()
              time_diff=get_time_ex(start,'_',end)
             
              information_schema.append(nameValue)
              information_schema.append(nameField==fields[j])
              information_schema.append(time_diff)
              
              print("XPAth:", nameFieldXpath)
              print("VAL XPath", nameValueXpath)
              print('Nome:',nameField)
              print('valore:',nameValue)
              print('atteso:',fields[j])
              print('corretto', correct)
    

      print(information_schema)
      return information_schema

              
          
          
       
          

In [ ]:
def extract_info_for_USA(link,id):
     
    
  x_paths_W_tradeFields=[
              ('//*[@id="company_identity"]/div[1]/div[1]/span/span','//*[@id="company_identity"]/div[1]/div[1]/span/strong'),
             ('//*[@id=\"company_identity\"]/div[1]/div[2]/span/span','//*[@id=\"company_identity\"]/div[1]/div[2]/span/text()'),
             
             ('//*[@id=\"company_identity\"]/div[1]/div[3]/span/span','//*[@id=\"company_identity\"]/div[1]/div[3]/span/text()'),
             
             ('//*[@id=\"company_identity\"]/div[1]/div[4]/span/span','//*[@id=\"company_identity\"]/div[1]/div[4]/span/text()'),
             ('//*[@id=\"company_identity\"]/div[1]/div[5]/span/span','//*[@id=\"company_identity\"]/div[1]/div[5]/span/text()'),
             ('//*[@id=\"company_identity\"]/div[1]/div[6]/span/span','//*[@id=\"company_identity\"]/div[1]/div[6]/span/text()'),
             ('//*[@id=\"company_identity\"]/div[1]/div[7]/span/span','//*[@id="company_identity"]/div[1]/div[7]/span/a'),
             ('//*[@id=\"company_identity\"]/div[1]/div[8]/span/span','//*[@id=\"company_identity\"]/div[1]/div[8]/span/text()')]
             
  x_paths_no_tradeFields=[
            ('//*[@id="company_identity"]/div[1]/div[1]/span/span','//*[@id="company_identity"]/div[1]/div[1]/span/strong'),
             ('//*[@id=\"company_identity\"]/div[1]/div[2]/span/span','//*[@id=\"company_identity\"]/div[1]/div[2]/span/text()'),
             
             ('//*[@id=\"company_identity\"]/div[1]/div[3]/span/span','//*[@id=\"company_identity\"]/div[1]/div[3]/span/text()'),
            
             ('//*[@id=\"company_identity\"]/div[1]/div[4]/span/span','//*[@id=\"company_identity\"]/div[1]/div[4]/span/text()'),
             ('//*[@id=\"company_identity\"]/div[1]/div[5]/span/span','//*[@id=\"company_identity\"]/div[1]/div[5]/span/text()'),
             ('//*[@id=\"company_identity\"]/div[1]/div[6]/span/span','//*[@id="company_identity"]/div[1]/div[6]/span/a'),
             ('//*[@id=\"company_identity\"]/div[1]/div[7]/span/span','//*[@id=\"company_identity\"]/div[1]/div[7]/span/text()')]

    
    #i fields assocaiti
    #servono per un check
  fields_W_tradeField=['Name','Trade Name', 'Address', 'City', 'State', 'Region', 'Country','Location Type']
  fields_no_tradeField=['Name', 'Address', 'City', 'State', 'Region', 'Country','Location Type']
    
    
  information_schema=[]
    
  start = datetime.now()
  req = requests.get(link, headers)
  end = datetime.now()
  time_diff=get_time_ex(start,'access',end)
  soup = BeautifulSoup(req.content, 'html.parser')
  dom = etree.HTML(str(soup))
  contains_trade_name=False
  #verifico la presenza di State
  xPathColumn, _=x_paths_W_tradeFields[3]
  xPathColumn2,_=x_paths_W_tradeFields[4]


  nameField=dom.xpath(xPathColumn)[0].text
  nameField2=dom.xpath(xPathColumn2)[0].text

  print(nameField)
  print(nameField2)

  if ('State' in nameField or 'State' in nameField2):
      #verifico la presenza di Trade Name
      xPathColumn, _=x_paths_W_tradeFields[1]
      nameField=dom.xpath(xPathColumn)[0].text
      print(nameField)
      if 'Trade' in nameField:
        x_paths=x_paths_W_tradeFields
        fields=fields_W_tradeField
        infos=extractDataWithNameField(dom,x_paths, fields,link,id, information_schema)
        print(infos)
        
      else:
        x_paths=x_paths_no_tradeFields
        fields=fields_no_tradeField
        
        infos=extractDataWithNoNameField(dom,x_paths, fields,link,id,information_schema)
      
        print(infos)

  return information_schema
 

     
   
    

In [ ]:
from csv import reader
from csv import writer
def scrape_data(path_dataset_link, path_dataset_aziende):
    
    with open(path_dataset_link, 'r') as read_obj:
   
        csv_reader = reader(read_obj)
    
        writer_a=open(path_dataset_aziende,'w')
        csv_writer=csv.writer(writer_a)
        id=1
        headers=['URL', 'ID', 'Name', 'corretto Name', 'tempo Estrazione Name',
             'Trade Name', 'corretto Trade Name', 'tempo estrazione Trade Name',
             'Address Name', 'corretto Address', 'tempo estrazione Address',
             'City', 'corretto City', 'tempo estrazione City',
             'Postalcode',
             'State', 'corretto State', 'tempo estrazione State',
             'Country', 'corretto Country', 'tempo estrazione Country',
             'Location type', 'corretto location type', 'tempo estrazione Location Type',
             ]
    
        csv_writer.writerow(headers)
        print('Printed headers')
        conta=0
    
        # Iterate over each row in the csv using reader object
        for row in csv_reader:
            print(row)
            print(conta)
            if conta>0:
                # row variable is a list that represents a row in csv
                
                for j in range (0,4):
          
                    if j==0:
                        link=row[j]
                        print(link)
                        infos=extract_info_for_USA(link,id)
                    #print(infos)
                    else:
                            if row[j]!='':
                                link=row[j]
                                infos=extract_info_for_other_countries(link,id)
                    print(infos)
            
            
                if len(infos)>0:    
                    csv_writer.writerow(infos)
                    id+=1
                
            conta+=1 
        writer_a.close()         
            
          


In [ ]:
from csv import reader
from csv import writer
def scrape_data(path_dataset_link, path_dataset_aziende):
    
    with open(path_dataset_link, 'r') as read_obj:
   
        csv_reader = reader(read_obj)
    
        writer_a=open(path_dataset_aziende,'w')
        csv_writer=csv.writer(writer_a)
        id=1
        headers=['URL', 'ID', 'Name', 'corretto Name', 'tempo Estrazione Name',
             'Trade Name', 'corretto Trade Name', 'tempo estrazione Trade Name',
             'Address Name', 'corretto Address', 'tempo estrazione Address',
             'City', 'corretto City', 'tempo estrazione City',
             'Postalcode',
             'State', 'corretto State', 'tempo estrazione State',
             'Country', 'corretto Country', 'tempo estrazione Country',
             'Location type', 'corretto location type', 'tempo estrazione Location Type',
             ]
    
        csv_writer.writerow(headers)
        print('Printed headers')
        conta=0
    
        # Iterate over each row in the csv using reader object
        for row in csv_reader:
            print('Riga file: ',row)
            print('Conteggio: ',conta)
            if conta>0:
                # row variable is a list that represents a row in csv
                
                for j in range (0,4):
                
                ''' if j==0:
                        print('usa')
                        link=row[j]
                        print('Link: ',link)
                        infos=extract_info_for_USA(link,id)
                        if len(infos)>0:    
                          csv_writer.writerow(infos)
                          print('PRINTED ELEMENT')
                          id+=1'''
                
                    
                    if j>0:
                            print('Other country')
                            if row[j]!='':
                                link=row[j]
                                infos=extract_info_for_other_countries(link,id)
                                if len(infos)>0:    
                                    csv_writer.writerow(infos)
                                    print("PRINTED ELEMENT")
                                    id+=1
                    print('Information scraped: ',infos)
                    print('Len:', len(infos))
            
            
                
                
            conta+=1 
        writer_a.close()         
            
          
